In [54]:
import pandas as pd

spatial = pd.read_csv('data/grunnkrets_norway_stripped.csv')
age = pd.read_csv('data/grunnkrets_age_distribution.csv')
income = pd.read_csv('data/grunnkrets_income_households.csv').set_index(['grunnkrets_id', 'year']).add_prefix('income_').reset_index()
households = pd.read_csv('data/grunnkrets_households_num_persons.csv')
submission = pd.read_csv('data/sample_submission.csv')
plaace = pd.read_csv('data/plaace_hierarchy.csv')
busstops = pd.read_csv('data/busstops_norway.csv')

train = pd.read_csv('data/stores_train.csv').drop(columns=['year'])
test = pd.read_csv('data/stores_test.csv') 

In [55]:
train.isna().sum()

store_id                   0
store_name                 0
plaace_hierarchy_id        0
sales_channel_name         0
grunnkrets_id              0
address                 1774
lat                        0
lon                        0
chain_name              9122
mall_name              10579
revenue                    0
dtype: int64

In [56]:
train_house = train.merge(households.drop(columns=['year']), on='grunnkrets_id', how='left')
train_spatial = train.merge(spatial.drop(columns=['year']), on='grunnkrets_id', how='left')
train_income = train.merge(income.drop(columns=['year']), on='grunnkrets_id', how='left')

train_house_no_nan = train_house[pd.notnull(train_house.couple_children_0_to_5_years)]
train_spatial_no_nan = train_spatial[pd.notnull(train_spatial.grunnkrets_name)]
train_income_no_nan = train_income[pd.notnull(train_income.income_all_households)]

In [59]:
for df in [train_house_no_nan, train_spatial_no_nan, train_income_no_nan]:
    cols_to_use = df.columns.difference(train.columns)
    train = train.merge(df[['grunnkrets_id', *cols_to_use]], on='grunnkrets_id', how='left')

train.head()

KeyError: 'grunnkrets_id'